# DHIS2 Analytics API - Using Training.DHIS2

In [42]:
# Important Packages

import pandas as pd
import requests
import json
from pathlib import Path
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta
import logging
import ast

### Enable Logging

In [43]:
# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

### Set Directories

In [44]:
# Current notebook's directory
base_dir = Path.cwd().parent  # Moves one level up from current working directory

# Paths
credentials_path = base_dir / '00_Local' / '01_Configs' / 'credentials.txt'
data_dir = base_dir / '00_Local' / '02_Data'
data_elements_csv = Path.cwd().parent / '00_Local' / '02_Data' / 'nd2_data_elements_ids.csv'
org_units_file = data_dir / 'organisation_units.csv'
districts_file = data_dir / 'districts.csv'
provinces_file = data_dir / 'provinces.csv'

### Load credentials from text file

In [45]:
# Load credentials from the file
credentials = {}
with credentials_path.open('r') as file:
    for line in file:
        if '=' in line:
            key, value = line.strip().split('=', 1)
            credentials[key.strip()] = value.strip()

# Extract values
DHIS2_URL = credentials.get('DHIS2_URL')
USERNAME = credentials.get('USERNAME')
PASSWORD = credentials.get('PASSWORD')

print("DHIS2 instance URL:", DHIS2_URL)

DHIS2 instance URL: https://training.eidsr.znphi.co.zm/


### Query DHIS2 Analytics API for available data sets.

In [46]:
# My API endpoint for datasets
endpoint = f"{DHIS2_URL}/api/dataSets.json"

# My request with authentication
response = requests.get(endpoint, auth=(USERNAME, PASSWORD), params={'paging': 'false'})

# Stepwise check
if response.status_code == 200:
    datasets = response.json()
    print("Data retrieved successfully!")
else:
    print("Failed to retrieve data:", response.status_code, response.text)


Data retrieved successfully!


### View the Retrieved Data in a Readable Format

In [47]:
# # Pretty-print the JSON response to understand the contents of the dictionary
# print(json.dumps(datasets, indent=4))

### Convert the Data to a Pandas DataFrame

In [48]:
# Convert JSON to DataFrame
df = pd.DataFrame(datasets['dataSets'])

In [49]:
df.head(20)

,displayName,id
0,Cholera Vaccination,xoR1fw9UAnC
1,Community EBS,GWpIuhj6AEI
2,Community EBS Redesign,nkH6V3U9bJ8
3,COVID-19 Daily Monitoring and Contact Tracing,X2vykx4vd8C
4,COVID-19 Homecare Report,dBM7yaDxy9g
5,COVID-19 Laboratory Daily Aggregated Tool,gOr8UcpIqLA
6,COVID-19 Surveillance Daily Aggregate Report,Ctvbrr4atAn
7,COVID Cases Management Daily Aggregate Report,dTFXTfmgjgv
8,HF-IDSR Core Function Assessment Tool,SA4ANhumZ9v
9,ND2 Report,BtzVmjyJ6DZ


In [50]:
# Save Data for Future Analysis

df.to_csv(data_dir / 'datasets.csv', index=False)
print("Datasets saved to datasets.csv")

Datasets saved to datasets.csv


### View All Organisation Units

In [51]:
# API endpoint for organisation units
org_units_url = f"{DHIS2_URL}/api/organisationUnits.json"

# Parameters to retrieve all units without paging
params = {
    'paging': 'false',  # Retrieve all records without pagination
    'fields': 'id,displayName,level,parent[id,displayName]'  # Fetch necessary fields
}

# Request
response = requests.get(org_units_url, auth=(USERNAME, PASSWORD), params=params)

# Process the response
if response.status_code == 200:
    org_units = response.json()
    print("Organisation Units retrieved successfully!")
else:
    print("Error fetching organisation units:", response.status_code, response.text)


Organisation Units retrieved successfully!


In [52]:
# print(json.dumps(org_units, indent=4))

In [53]:
# Extract the organisation unit details
org_units_list = org_units.get('organisationUnits', [])

# Convert to DataFrame
df_org_units = pd.DataFrame(org_units_list)

In [54]:
df_org_units

,parent,displayName,id,level
0,"{'displayName': 'ea Chasefu District', 'id': '...",ae Chasefu Mini Hospital,qt4781BQa2L,4
1,"{'displayName': 'ls Chilanga District', 'id': ...",Balmoral Health Post,FrOn8lcbzPb,4
2,"{'displayName': 'ce Luano DIstrict', 'id': 'hP...",ce Bbusa Health Post,EPtuFlHv12i,4
3,"{'displayName': 'ce Kabwe District', 'id': 'sN...",ce Blossom Surgery Clinic,S9tqzMHVdMU,4
4,"{'displayName': 'ce Chisamba District', 'id': ...",ce Bombwe Health post,FIU3dK7BniN,4
...,...,...,...,...
3827,"{'displayName': 'ls Chilanga District', 'id': ...",ZA-Apollo Rural Health Centre,sfSGuVk954T,4
3828,"{'displayName': 'ls Chilanga District', 'id': ...",Zambia Helpers Society Hospital,oqBLYbMJXcg,4
3829,"{'displayName': 'ce Chibombo District', 'id': ...",zm Test Facility,BU7fjlCqw4o,4
3830,NaN,zm Zambia Ministry of Health,PS5JpkoHHio,1


#### Definition of Levels

In [55]:
# API endpoint for organisation unit levels (national, provincial, district etc...)
org_unit_levels_url = f"{DHIS2_URL}/api/organisationUnitLevels.json"

# Parameters to retrieve all levels without paging
params = {
    'paging': 'false',  # Retrieve all records without pagination
    'fields': 'id,level,displayName'  # Fetch necessary fields
}

# Request
response = requests.get(org_unit_levels_url, auth=(USERNAME, PASSWORD), params=params)

# Process the response
if response.status_code == 200:
    org_unit_levels = response.json()
    print("Organisation Unit Levels retrieved successfully!")
else:
    print("Error fetching organisation unit levels:", response.status_code, response.text)


Organisation Unit Levels retrieved successfully!


In [56]:
# Extract the organisation unit levels
org_levels_list = org_unit_levels.get('organisationUnitLevels', [])

# Convert to DataFrame
df_org_levels = pd.DataFrame(org_levels_list)

df_org_levels = df_org_levels.sort_values(by='level', ascending=True)

df_org_levels

,level,displayName,id
2,1,National,qXnFgkCQcRC
4,2,Province,HDS8ZqmIuJh
0,3,District,Dz7Sm3imvLU
1,4,Health Facilities,MQLiogB9XBV
3,5,NHC-Zones,R2H4dpJ4PSF


In [57]:
df_org_levels.to_csv(data_dir / 'organisation_units.csv', index=False)
print("Organisation_units saved to organisation_units.csv")

Organisation_units saved to organisation_units.csv


### Provinces

In [58]:
# Filter only province-level organisation units (level 2 as shown above)
df_provinces = df_org_units[df_org_units['level'] == 2]

# Province IDs as a comma-separated string
province_unit_ids = ",".join(df_provinces['id'])

print("Total provinces found:", len(df_provinces))
print("Province unit IDs:", province_unit_ids)

Total provinces found: 10
Province unit IDs: AWn3s2RqgAN,utIjliUdjp8,J7PQPWAeRUk,KozcEjeTyuD,B1u1bVtIA92,dbTLdTi7s8F,SwwuteU1Ajk,q5hODNmn021,oPLMrarKeEY,g1bv2xjtV0w


In [59]:
# Levels in the dataset
df_org_units['level'].unique()

# Ensure the correct level is being used for provinces (assumed level 2)
df_provinces = df_org_units[df_org_units['level'] == 2]

# Check if provinces were correctly filtered
df_provinces[['displayName', 'id', 'level']]

,displayName,id,level
8,ce Central Province,AWn3s2RqgAN,2
457,co Copperbelt Province,utIjliUdjp8,2
990,ea Eastern Province,J7PQPWAeRUk,2
1515,ls Lusaka Province,KozcEjeTyuD,2
1837,lu Luapula Province,B1u1bVtIA92,2
2157,mu Muchinga Province,dbTLdTi7s8F,2
2550,no Northern Province,SwwuteU1Ajk,2
2876,nw North-Western Province,q5hODNmn021,2
3442,so Southern Province,oPLMrarKeEY,2
3819,we Western Province,g1bv2xjtV0w,2


In [60]:
province_unit_ids = ";".join([
    "AWn3s2RqgAN",  # Central Province
    "utIjliUdjp8",  # Copperbelt Province
    "J7PQPWAeRUk",  # Eastern Province
    "KozcEjeTyuD",  # Lusaka Province
    "B1u1bVtIA92",  # Luapula Province
    "dbTLdTi7s8F",  # Muchinga Province
    "SwwuteU1Ajk",  # Northern Province
    "q5hODNmn021",  # North-Western Province
    "oPLMrarKeEY",  # Southern Province
    "g1bv2xjtV0w"   # Western Province
])

print("Formatted Province unit IDs:", province_unit_ids)


Formatted Province unit IDs: AWn3s2RqgAN;utIjliUdjp8;J7PQPWAeRUk;KozcEjeTyuD;B1u1bVtIA92;dbTLdTi7s8F;SwwuteU1Ajk;q5hODNmn021;oPLMrarKeEY;g1bv2xjtV0w


In [74]:
# Extract and save districts (Level 3)
df_districts = df_org_units[df_org_units['level'] == 3][['id', 'displayName', 'parent']]
df_districts.to_csv(districts_file, index=False)
print(f"Districts saved to districts.csv")

# Extract and save provinces (Level 2)
df_provinces = df_org_units[df_org_units['level'] == 2][['id', 'displayName', 'parent']]
df_provinces.to_csv(provinces_file, index=False)
print(f"Provinces saved to provinces.csv")

Districts saved to districts.csv
Provinces saved to provinces.csv


### QUERY ND2

In [61]:
# ND2 Report dataset ID
nd2_report_id = "BtzVmjyJ6DZ"

# API endpoint
nd2_report_url = f"{DHIS2_URL}/api/dataSets/{nd2_report_id}.json"

# Parameters to retrieve metadata (data elements, category combos, etc.)
params = {
    'fields': 'id,displayName,dataSetElements[dataElement[id,displayName,valueType]],periodType',
    'paging': 'false'
}

# Request
response = requests.get(nd2_report_url, auth=(USERNAME, PASSWORD), params=params)

# Check if the request was successful
if response.status_code == 200:
    nd2_report_data = response.json()
    print("ND2 Report metadata retrieved successfully!")
else:
    print("Error fetching ND2 Report data:", response.status_code, response.text)


ND2 Report metadata retrieved successfully!


In [62]:
# print(json.dumps(nd2_report_data, indent=4))

In [63]:
# Extract data elements
data_elements = nd2_report_data.get('dataSetElements', [])

# Convert to DataFrame
df_data_elements = pd.DataFrame([
    {
        'dataElement_id': element['dataElement']['id'],
        'dataElement_name': element['dataElement']['displayName'],
        'valueType': element['dataElement']['valueType']
    }
    for element in data_elements
])

In [64]:
df_data_elements.head(20)

,dataElement_id,dataElement_name,valueType
0,LEFTRuBi3xe,Dysentery,NUMBER
1,o5nSN7w2ILg,Measles sent to lab,NUMBER
2,wfS6S7jMCoq,Rabies sent to lab,NUMBER
3,dMt4DePB5Hx,SARS sent to Lab,NUMBER
4,e9tQpS8rxAw,Influenza (New) suspected,NUMBER
5,K91L3VNWN1C,Trypanosomiasis death,NUMBER
6,S7p6aVSXvdD,Food Poisoning death,NUMBER
7,WLIZlT649SW,MDR/XDR Tuberculosis death,NUMBER
8,D2ThDRykD5G,Chicken Pox death,NUMBER
9,EKWw4vPCtIQ,COVID-19 sent to lab,NUMBER


In [65]:
df_data_elements.to_csv(data_dir / 'data_elements.csv', index=False)
print("ND2 Data Elements saved to df_data_elements.csv")


ND2 Data Elements saved to df_data_elements.csv


## ND2 Cholera Data

### Cholera Data Elements

In [66]:
# Filter rows where 'dataElement_name' contains 'Cholera'
cholera_data_elements = df_data_elements[df_data_elements['dataElement_name'].str.contains('Cholera', case=False, na=False)]

# Display filtered elements
cholera_data_elements


,dataElement_id,dataElement_name,valueType
20,gMy3ugmaS8z,Cholera sent to lab,NUMBER
66,norzLTZKFeO,Cholera suspected,NUMBER
87,AzkTqGTlreJ,Cholera confirmed,NUMBER
96,Ay3ZqJMukSP,Cholera death,NUMBER


In [67]:
cholera_data_elements.to_csv(data_dir / 'cholera_data_elements.csv', index=False)
print("Cholera data elements saved to cholera_data_elements.csv")

Cholera data elements saved to cholera_data_elements.csv


In [68]:
cholera_element_ids = ";".join(cholera_data_elements['dataElement_id'])

print("Cholera Data Element IDs:", cholera_element_ids)

Cholera Data Element IDs: gMy3ugmaS8z;norzLTZKFeO;AzkTqGTlreJ;Ay3ZqJMukSP


### Monthly Cholera Cases

In [69]:
# DHIS2 API URL for analytics data
analytics_url = f"{DHIS2_URL}/api/analytics.json"

# Cholera element IDs from the dataset
cholera_element_ids = "Ay3ZqJMukSP;norzLTZKFeO;gMy3ugmaS8z;AzkTqGTlreJ"

# Define the period of interest (January 2024 to December 2024)
period_range = ";".join([pd.to_datetime(date).strftime('%Y%m') for date in pd.date_range(start="2024-01-01", end="2024-12-31", freq='MS')])

# Define national-level organisation unit (Ministry of Health)
national_org_unit_id = "PS5JpkoHHio"

# Construct API query parameters
params = {
    'dimension': [
        f'dx:{cholera_element_ids}',  # Cholera data element IDs
        f'pe:{period_range}',  # Monthly periods for 2024
        f'ou:{national_org_unit_id}'  # National org unit
    ],
    'displayProperty': 'NAME'
}

# Request data from DHIS2 API
response = requests.get(analytics_url, auth=(USERNAME, PASSWORD), params=params)

# Check response status
if response.status_code == 200:
    cholera_data = response.json()
    print("Cholera data retrieved successfully!")
else:
    raise Exception(f"Error retrieving Cholera data: {response.status_code}")


Cholera data retrieved successfully!


In [70]:
# Convert API response to DataFrame
if 'rows' in cholera_data:
    df_cholera = pd.DataFrame(cholera_data['rows'], columns=['dataElement', 'period', 'orgUnit', 'value'])

    # Convert period to readable format (YYYYMM to Month-Year)
    df_cholera['Date'] = pd.to_datetime(df_cholera['period'], format='%Y%m').dt.strftime('%b-%Y')

    # Map element IDs to their respective names
    cholera_mapping = {
        "Ay3ZqJMukSP": "Cholera death",
        "norzLTZKFeO": "Cholera suspected",
        "gMy3ugmaS8z": "Cholera sent to lab",
        "AzkTqGTlreJ": "Cholera confirmed"
    }

    df_cholera['dataElement'] = df_cholera['dataElement'].map(cholera_mapping)

    # Convert 'value' column to numeric and ensure integer values
    df_cholera['value'] = pd.to_numeric(df_cholera['value'], errors='coerce').fillna(0).astype(int)

    # Pivot table to restructure the data
    df_cholera_pivot = df_cholera.pivot_table(index='Date', columns='dataElement', values='value', aggfunc='sum').reset_index()

    # Ensure correct column ordering and fill missing columns with 0
    required_columns = ['Date', 'Cholera suspected', 'Cholera sent to lab', 'Cholera death', 'Cholera confirmed']

    for col in required_columns:
        if col not in df_cholera_pivot.columns:
            df_cholera_pivot[col] = 0  # Fill missing columns with 0

    df_cholera_pivot = df_cholera_pivot[required_columns]

    # Convert Date column for sorting
    df_cholera_pivot['Date'] = pd.to_datetime(df_cholera_pivot['Date'], format='%b-%Y')

    # Sort by date and convert it back to string
    df_cholera_pivot = df_cholera_pivot.sort_values(by='Date').reset_index(drop=True)
    df_cholera_pivot['Date'] = df_cholera_pivot['Date'].dt.strftime('%b-%Y')

    # Convert all numerical columns to integers to remove decimals
    for col in df_cholera_pivot.columns:
        if col != 'Date':  # Exclude Date column from conversion
            df_cholera_pivot[col] = df_cholera_pivot[col].fillna(0).astype(int)

else:
    print("No Cholera data found for the selected period.")


In [71]:
df_cholera_pivot.head(12)

dataElement,Date,Cholera suspected,Cholera sent to lab,Cholera death,Cholera confirmed
0,Jan-2024,9117,1844,132,719
1,Feb-2024,4791,1697,31,1038
2,Mar-2024,1542,398,7,114
3,Apr-2024,751,213,12,38
4,May-2024,197,142,1,55
5,Jun-2024,133,70,0,99
6,Jul-2024,27,24,0,6
7,Aug-2024,45,5,0,0
8,Sep-2024,141,1,0,0
9,Oct-2024,49,9,0,0


## Facility-Level Weekly Data

### Proof of Concept: January Only

In [75]:
# Read data elements from dataframe
dx_ids = ';'.join(df_data_elements['dataElement_id'].tolist())

# Use facility level ID
facility_level_id = "MQLiogB9XBV"

# Parse parent column to extract IDs from string representation
def parse_parent_column(df):
    """ Extract parent ID from string dictionary in the 'parent' column. """
    df['parent'] = df['parent'].apply(lambda x: ast.literal_eval(x)['id'] if pd.notnull(x) else None)
    return df

# Fetch facility-level organisation units from DHIS2
def fetch_facilities():
    """Fetch all facilities (level 4) from DHIS2 and store them in a dataframe."""
    url = f"{DHIS2_URL}/api/organisationUnits.json"
    params = {"filter": "level:eq:4", "paging": "false", "fields": "id,displayName,parent[id]"}

    try:
        response = requests.get(url, params=params, auth=HTTPBasicAuth(USERNAME, PASSWORD))
        response.raise_for_status()
        facilities_data = response.json()
        facilities_df = pd.DataFrame(facilities_data['organisationUnits'])

        # Parse the parent ID from dictionary and rename displayName to facility_name
        facilities_df['parent'] = facilities_df['parent'].apply(lambda x: x['id'] if pd.notnull(x) else None)
        facilities_df.rename(columns={'displayName': 'facility_name'}, inplace=True)

        logger.info("Facilities data fetched successfully.")
        return facilities_df

    except Exception as e:
        logger.error(f"Error fetching facility data: {e}")
        return None

# Load organisation unit data for districts and provinces
def load_organisation_units():
    """ Load and clean organisation units. """
    try:
        districts = pd.read_csv(districts_file)
        provinces = pd.read_csv(provinces_file)

        # Extract parent IDs from string dictionary format
        districts = parse_parent_column(districts)
        provinces = parse_parent_column(provinces)

        logger.info("Organisation units loaded and cleaned successfully.")
        return districts, provinces
    except FileNotFoundError as e:
        logger.error(f"Organisation units file not found: {e}")
        return None, None
    except Exception as e:
        logger.error(f"Error loading organisation units: {e}")
        return None, None

# Convert week string to date
def week_to_date(week_str):
    year, week = int(week_str[:4]), int(week_str[5:])
    first_day_of_year = datetime(year, 1, 1)
    first_week_start = first_day_of_year - timedelta(days=first_day_of_year.weekday())
    return first_week_start + timedelta(weeks=week - 1)

# Fetch and process DHIS2 analytics data
def fetch_and_transform_data():
    logger.info("Fetching facility-level data from DHIS2...")
    
    districts, provinces = load_organisation_units()
    facilities_df = fetch_facilities()
    
    if districts is None or provinces is None or facilities_df is None:
        return None

    url = f'{DHIS2_URL}/api/analytics.json'
    params = {
        "dimension": f"dx:{dx_ids},pe:2024W01;2024W02;2024W03;2024W04,ou:LEVEL-{facility_level_id}"
    }

    try:
        response = requests.get(url, params=params, auth=HTTPBasicAuth(USERNAME, PASSWORD))
        response.raise_for_status()
        logger.info("Facility-level data fetched successfully.")
        data = response.json()
    except requests.exceptions.HTTPError as err:
        logger.error(f"HTTP error: {err}")
        return None
    except requests.exceptions.RequestException as err:
        logger.error(f"Request error: {err}")
        return None

    try:
        headers = data.get('headers', [])
        rows = data.get('rows', [])
        columns = [header['name'] for header in headers]
        df = pd.DataFrame(rows, columns=columns)

        df['date'] = df['pe'].apply(week_to_date)
        df.sort_values('date', ascending=False, inplace=True)

        df['value'] = pd.to_numeric(df['value'], downcast='integer', errors='coerce')
        df = df.merge(df_data_elements, left_on='dx', right_on='dataElement_id', how='left')
        df['dx'] = df['dataElement_name']
        df.drop(['dataElement_name', 'dataElement_id'], axis=1, inplace=True)

        # Merge facility with districts and include facility name
        df = df.merge(facilities_df[['id', 'facility_name', 'parent']], left_on='ou', right_on='id', how='left')
        df.rename(columns={'id': 'facility_id', 'parent': 'district_id'}, inplace=True)

        df = df.merge(districts[['id', 'displayName', 'parent']], left_on='district_id', right_on='id', how='left')
        df.rename(columns={'displayName': 'district_name', 'parent': 'province_id'}, inplace=True)

        # Merge districts with provinces
        df = df.merge(provinces[['id', 'displayName']], left_on='province_id', right_on='id', how='left')
        df.rename(columns={'displayName': 'province_name'}, inplace=True)

        # Pivot data including the new columns
        pivoted_df = df.pivot_table(index=['pe', 'facility_id', 'district_id', 'province_id', 'facility_name', 'district_name', 'province_name', 'date'], 
                                columns='dx', 
                                values='value', 
                                aggfunc='first')
        
        pivoted_df.reset_index(inplace=True)
        pivoted_df.fillna(0, inplace=True)
        pivoted_df.rename(columns={'pe': 'period'}, inplace=True)

        # Convert columns to integer where applicable
        excluded_columns = ['dx', 'period', 'facility_id', 'district_id', 'province_id', 'facility_name', 'district_name', 'province_name', 'date']
        columns_to_convert = [col for col in pivoted_df.columns if col not in excluded_columns]
        for column in columns_to_convert:
            pivoted_df[column] = pd.to_numeric(pivoted_df[column], downcast='integer', errors='coerce')

        return pivoted_df

    except Exception as e:
        logger.error(f"Error processing data: {e}")
        return None


# Process the data and save to CSV
def process_data():
    logger.info("Processing and saving facility-level data with organisation details...")

    df = fetch_and_transform_data()
    if df is None:
        logger.error("Data processing failed. Skipping saving steps.")
        return

    output_file = data_dir / 'facility_january_2024_with_org_details.csv'
    df.to_csv(output_file, index=False)
    logger.info("Facility-level data with organisation details saved.")

# Execute processing function
process_data()

INFO:__main__:Processing and saving facility-level data with organisation details...
INFO:__main__:Fetching facility-level data from DHIS2...
INFO:__main__:Organisation units loaded and cleaned successfully.
INFO:__main__:Facilities data fetched successfully.
INFO:__main__:Facility-level data fetched successfully.
INFO:__main__:Facility-level data with organisation details saved.


In [76]:
facility_data_january = pd.read_csv(data_dir / 'facility_january_2024_with_org_details.csv')

In [77]:
facility_data_january.head(10)

,period,facility_id,district_id,province_id,facility_name,district_name,province_name,date,AEFI sent to lab,AEFI suspected,...,Trypanosomiasis confirmed,Trypanosomiasis sent to lab,Trypanosomiasis suspected,Tuberculosis confirmed,Tuberculosis death,Tuberculosis sent to lab,Tuberculosis suspected,Typhoid fever confirmed,Typhoid fever sent to Lab,Typhoid fever suspected
0,2024W1,A2SSdvZe7ur,svOkUE0zEKJ,q5hODNmn021,nw Lumwe Health Post,nw Mufumbwe District,nw North-Western Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2024W1,A2mkoh0Qoh1,PlQiD2woAPw,SwwuteU1Ajk,no Pecha Health Post,no Lunte District,no Northern Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2024W1,A4vc3yQR2An,SJjjM0qNelL,SwwuteU1Ajk,no Chikwanda Health Post,no Senga District,no Northern Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2024W1,A6Zdi9v8spx,WUGwOojuXhn,dbTLdTi7s8F,mu Mukungule Rural Health Centre,mu Mpika District,mu Muchinga Province,2024-01-01,0,0,...,0,0,0,0,0,0,24,0,0,0
4,2024W1,A6hsoKzGQh5,SQT8xjbvWwf,oPLMrarKeEY,so Muunga Rural Health Centre,so Itezhi-tezhi District,so Southern Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
5,2024W1,A6kk1S24wfa,DM5Sjew7MCn,oPLMrarKeEY,so Prison Urban Health Centre,so Livingstone District,so Southern Province,2024-01-01,0,0,...,0,0,0,0,0,29,29,0,0,0
6,2024W1,A8TmYeix1cv,H8e2u6ULtJn,g1bv2xjtV0w,we Mata Rural Health Centre,we Senanga District,we Western Province,2024-01-01,0,0,...,0,0,0,0,0,1,1,0,0,0
7,2024W1,A9BC0fz8cxX,SfFrfySzRgY,utIjliUdjp8,co Muchindushi Health Post,co Mpongwe District,co Copperbelt Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
8,2024W1,ABOFImxcg03,oEBf29y8JP8,AWn3s2RqgAN,ce Shimukuni Rural Health Centre,ce Chibombo District,ce Central Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
9,2024W1,ABcUyezNjiu,A84b9TjHfN4,B1u1bVtIA92,lu Mwansakombe Rural Health Centre,lu Chifunabuli District,lu Luapula Province,2024-01-01,0,0,...,0,0,0,0,0,0,0,0,0,0
